# 决策树ID3代码实现（Python语言）

In [13]:
from math import log
import numpy as np
import operator

<img src="images/01.png" width="300" align="left"/>

In [14]:
def loaddata():
    dataSet = [[0, 0,0,0, 'no'],
               [0, 0,0,1,'no'],
               [0, 1,0,1, 'yes'],
               [0, 1,1,0, 'yes'],
               [0, 0,0,0, 'no'],
               [1, 0,0,0, 'no'],
               [1, 0,0,1, 'no'],
               [1, 1,1,1, 'yes'],
               [1, 0,1,2, 'yes'],
               [1, 0,1,2, 'yes'],
               [2, 0,1,2, 'yes'],
               [2, 0,1,1, 'yes'],
               [2, 1,0,1, 'yes'],
               [2, 1,0,2, 'yes'],
               [2, 0,0,0,'no']]
    feature_name = ['age','job','house','credit']
    return dataSet, feature_name

In [ ]:
# for example in dataSet:
#     print(example)

In [ ]:
# #特征数
# n = len(dataSet[0])-1
# #遍历每个特征
# for i in range(n):  
#   #获取当前特征的所有值
#         featList = [example[i] for example in dataSet]   # 获取第i列特征的所有值
# #         print(featList)
#         #当前特征的可能取值
#         uniqueVals = set(featList)
# #         print(uniqueVals)

### 计算数据集的熵
<img src="images/02.png" width="200" align="left"/>

In [15]:
def entropy(dataSet):
    #数据集条数
    m = len(dataSet)
    
    #标签不同类别的计数字典
    labelCounts={}
    #循环数据集
    for featVec in dataSet:
        #取标签
        currentLabel = featVec[-1]
        #如果字典中不存在，则值为0，否则值加1
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel]+=1
    #保存最终的熵值
    e = 0.0 
    
    #根据公式计算熵
    for key in labelCounts:
#         print(key)
        prob = float(labelCounts[key])/m
        e -= prob * log(prob,2)
    return e

测试：以下代码的输出结果应该是：0.971

In [16]:
dataSet, feature_name = loaddata()
entropy(dataSet)

0.9709505944546686

### 划分数据集

In [17]:
# 从dataSet中取出第axis个特征的值为value样本，组成列表。去掉第第axis个特征。
def splitDataSet(dataSet, axis, value):
    #按轴和值划分好的数据集
    retDataSet = []
    #循环数据集
    for featVec in dataSet:
        #当前数据中，按轴取出第axis个特征的数值符合传入的value值的数据
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]   # 把前axis个特征的值，
            reducedFeatVec.extend(featVec[axis+1:])  # 把第axis+1个特征的值，保存
            retDataSet.append(reducedFeatVec)
    return retDataSet

<img src="images/04.png" width="150" height="50" align="left"/>

测试：以下代码的运行结果应该是
[[0, 0, 0, 'no'],
 [0, 0, 1, 'no'],
 [0, 1, 1, 'yes'],
 [0, 0, 0, 'no'],
 [1, 0, 0, 'no'],
 [1, 0, 1, 'no'],
 [2, 1, 1, 'yes'],
 [2, 1, 2, 'yes'],
 [2, 0, 0, 'no']]

In [18]:
splitDataSet(dataSet,2,0)

[[0, 0, 0, 'no'],
 [0, 0, 1, 'no'],
 [0, 1, 1, 'yes'],
 [0, 0, 0, 'no'],
 [1, 0, 0, 'no'],
 [1, 0, 1, 'no'],
 [2, 1, 1, 'yes'],
 [2, 1, 2, 'yes'],
 [2, 0, 0, 'no']]

### 选择最好的特征
<img src="images/03.png" width="400" align="left"/>

In [19]:
# 信息增益法选择最优划分属性
def chooseBestFeature(dataSet):
    #特征数
    n = len(dataSet[0])-1
    
    #计数整个数据集的熵
    baseEntropy = entropy(dataSet)
    
    bestInfoGain = 0.0; bestFeature = -1
    
    #遍历每个特征
    for i in range(n):  
        #获取当前特征的所有值
        featList = [example[i] for example in dataSet]  # 获取第i列特征的所有值
        #当前特征的可能取值
        uniqueVals = set(featList)
        
        #定义一临时变量保存当前的条件熵
        newEntropy = 0.0
        #循环每一个可能的取值
        for value in uniqueVals:
            #按该值进行数据集的划分
            subDataSet = splitDataSet(dataSet,i,value)
            
            #计算条件熵（2行代码）
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob*entropy(subDataSet)
            
        #计算信息增益
        infoGain = baseEntropy - newEntropy
        
        #保存当前最大的信息增益及对应的特征
        if (infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
    #返回最优特征
    return bestFeature

测试：运行下面的代码，正确返回值是：2，表示按第二个特征进行分裂。

In [20]:
chooseBestFeature(dataSet)

2

### 类别的投票表决

In [21]:
def classVote(classList):
    #定义一字典，记录每个标签对应的个数
    classCount = {}
    
    #循环计数
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] += 1
        
    #排序
    print('classCount.items()=',classCount.items())
    
    sortedClassCount = sorted(classCount.items(),key = operator.itemgetter(1),reverse=True)
    print('sortedClassCount',sortedClassCount)
    
    return sortedClassCount[0][0]

测试：下面代码的返回结果应该是：yes

In [22]:
classList= np.array(['yes','no','yes','no','yes'])
classVote(classList)

classCount.items()= dict_items([('yes', 3), ('no', 2)])
sortedClassCount [('yes', 3), ('no', 2)]


'yes'

### 递归训练一棵树

In [23]:
def trainTree(dataSet,feature_name):
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList): 
        return classList[0]  #所有类别都一致
    
    if len(dataSet[0]) == 1: #数据集中只剩下一个特征
        return classVote(classList)
    
    bestFeat = chooseBestFeature(dataSet)
    bestFeatName = feature_name[bestFeat]
    myTree = {bestFeatName:{}}   # 决策树的数据结构：嵌套的字典
    
    # del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    
    for value in uniqueVals:
        sub_feature_name = feature_name[:]
        myTree[bestFeatName][value] = trainTree(splitDataSet(dataSet, bestFeat, value),
                                                sub_feature_name)
    return myTree  

In [24]:
myDat,feature_name = loaddata()
myTree = trainTree(myDat,feature_name)
print(myTree)

{'house': {0: {'job': {0: 'no', 1: 'yes'}}, 1: 'yes'}}


### 预测

In [25]:
def predict(inputTree,feature_name,testVec):
    print('inputTree.keys()=',inputTree.keys())
    
    firstStr = list(inputTree.keys())[0]
    print('firstStr=',firstStr)
    
    secondDict = inputTree[firstStr]
    print('secondDict=',secondDict)
    
    featIndex = feature_name.index(firstStr)
    print('featIndex=',featIndex)
    
    key = testVec[featIndex]
    print('key=',key)
    
    valueOfFeat = secondDict[key]
    print('---------------------')
    
    if isinstance(valueOfFeat, dict): 
        classLabel = predict(valueOfFeat, feature_name, testVec)
    else: classLabel = valueOfFeat
        
    return classLabel

In [26]:
# 输出预测
print(predict(myTree,feature_name,[1,1,0,1]))   #  [1,1,0,1]为新样本

inputTree.keys()= dict_keys(['house'])
firstStr= house
secondDict= {0: {'job': {0: 'no', 1: 'yes'}}, 1: 'yes'}
featIndex= 2
key= 0
---------------------
inputTree.keys()= dict_keys(['job'])
firstStr= job
secondDict= {0: 'no', 1: 'yes'}
featIndex= 1
key= 1
---------------------
yes
